In [1]:
using CSV, DataFrames
# D = demand matrix (day of the week)
# n locations
# W = foot-traffic matrix (for each location, foot traffic for day of the week)

D = CSV.read("electricity/demand.csv", DataFrame; header=true)[:, 2]#7x1
mean_counts_df = CSV.read("ped_traffic/mean_counts.csv", DataFrame; header=true)
W = mean_counts_df[:, 3:9]|> Matrix #110x7
location_ids = mean_counts_df[:, 1] # location IDs

mean_counts_df2 = CSV.read("ped_traffic/mean_counts_dist.csv", DataFrame; header=true)
W2 = mean_counts_df2[:, 3:9]|> Matrix #44x7
d = mean_counts_df2[:, 10] #44x1
location_ids2 = mean_counts_df2[:, 1] # location IDs

println(size(D))
println(size(W))
println(size(W2))
println(size(d))

(7,)
(110, 7)
(44, 7)
(44,)


# Number of Locations Minimization Optimization

In [2]:
using JuMP
using HiGHS
using LinearAlgebra

percent_demand = 0.00001
watts_converted = 0.1

function optimize_installations(D::Vector{Float64}, W::Matrix{Float64})

    n, T = size(W)  # n locations, T = 7 days
    @assert length(D) == T "Demand vector D must have length 7"

    model = Model(HiGHS.Optimizer)
    
    set_silent(model)

    @variable(model, x[1:n], Bin) # int decision vars

    @objective(model, Min, sum(x[i] for i in 1:n)) # objective func

    for t in 1:T # constraint each day
        @constraint(model, sum(W[i, t] * x[i] * watts_converted for i in 1:n) ≥ percent_demand * D[t])
    end

    optimize!(model) # solve model

    status = termination_status(model)
    if status != MOI.OPTIMAL
        println("Warning: Solver returned status $status")
    end

    chosen_locations = findall(i -> value(x[i]) > 0.5, 1:n)
    min_num_locations = length(chosen_locations)

    return min_num_locations, chosen_locations, model
end

optimize_installations (generic function with 1 method)

In [3]:
min_locs, chosen, model = optimize_installations(D, W)
println("Minimum number of locations needed: ", min_locs)
println("Chosen location indices: ", chosen)

Minimum number of locations needed: 4
Chosen location indices: [15, 17, 19, 78]


In [4]:
min_locs, chosen, model = optimize_installations(D, W2)
println("Minimum number of locations needed: ", min_locs)
println("Chosen location indices: ", chosen)

Minimum number of locations needed: 4
Chosen location indices: [6, 10, 20, 24]


In [5]:
# Calculate energy produced vs needed for each day
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
energy_produced = zeros(7)
energy_needed = percent_demand .* D

println(chosen)

for t in 1:7
    for i in chosen
        energy_produced[t] += W[i, t] * 0.1
        # energy_produced[t] += W2[i, t] * 0.1
    end
end

# Create comparison DataFrame
energy_comparison = DataFrame(
    Day = days,
    Energy_Produced = energy_produced,
    Energy_Needed = energy_needed,
    Surplus = energy_produced .- energy_needed,
    # Coverage_Percent = (energy_produced ./ energy_needed) .* 100
)

println("=" ^ 70)
println("Energy Production vs Demand Analysis")
println("=" ^ 70)
println(energy_comparison)
println()


[6, 10, 20, 24]
Energy Production vs Demand Analysis
7×4 DataFrame
 Row │ Day        Energy_Produced  Energy_Needed  Surplus  
     │ String     Float64          Float64        Float64  
─────┼─────────────────────────────────────────────────────
   1 │ Monday              415.45        1144.2   -728.754
   2 │ Tuesday             415.45        1181.74  -766.295
   3 │ Wednesday           415.45        1191.14  -775.693
   4 │ Thursday            415.45        1190.01  -774.564
   5 │ Friday              415.45        1161.43  -745.984
   6 │ Saturday            411.9         1054.46  -642.559
   7 │ Sunday              419.0         1061.85  -642.848



In [6]:
# Calculate total foot traffic per location (sum across all days)
total_foot_traffic = [sum(W[i, :]) for i in 1:size(W, 1)]

println(chosen)

# Create DataFrame with indices and total foot traffic
location_traffic = DataFrame(
    Index = 1:size(W, 1),
    Total_Foot_Traffic = total_foot_traffic
)

# Sort by total foot traffic (descending)
sort!(location_traffic, :Total_Foot_Traffic, rev=true)

println("=" ^ 70)
println("Top 10 Locations by Total Foot Traffic")
println("=" ^ 70)
println(first(location_traffic, 10))

[6, 10, 20, 24]
Top 10 Locations by Total Foot Traffic
10×2 DataFrame
 Row │ Index  Total_Foot_Traffic 
     │ Int64  Float64            
─────┼───────────────────────────
   1 │    15            34608.0
   2 │    20            24220.0
   3 │    19            20345.5
   4 │     9            18670.8
   5 │    78            16989.0
   6 │    75            14568.8
   7 │    17            14199.5
   8 │    31            14115.6
   9 │    57            11077.5
  10 │    58             9805.69


# Distance Minimization Optimization

In [27]:
using JuMP
using HiGHS
using LinearAlgebra

percent_demand = 0.00001
watts_converted = 0.1

function optimize_installations_dist(D::Vector{Float64}, W::Matrix{Float64}, d::Vector{Float64}, max_loc)

    n, T = size(W)  # n locations, T = 7 days
    @assert length(D) == T "Demand vector D must have length 7"
    @assert length(d) == n "Distance vector d must have length n"

    model = Model(HiGHS.Optimizer)
    
    set_silent(model)

    @variable(model, x[1:n], Bin) # int decision vars

    # Objective: minimize e^distance - 1 for selected locations
    @objective(model, Min, sum((d[i]*d[i] + d[i]) * x[i] for i in 1:n))

    # Constraint: maximum locations can be selected
    @constraint(model, sum(x[i] for i in 1:n) ≤ max_loc)

    # Constraint: energy production must meet demand each day
    for t in 1:T
        @constraint(model, sum(W[i, t] * x[i] * watts_converted for i in 1:n) ≥ percent_demand * D[t])
    end

    optimize!(model) # solve model

    status = termination_status(model)
    if status != MOI.OPTIMAL
        println("Warning: Solver returned status $status")
    end

    chosen_locations = findall(i -> value(x[i]) > 0.5, 1:n)
    total_distance = sum(d[i] for i in chosen_locations)
    num_locations = length(chosen_locations)

    return num_locations, chosen_locations, total_distance, model
end

optimize_installations_dist (generic function with 2 methods)

In [ ]:
num_locs, chosen, total_dist, model = optimize_installations_dist(D, W2, vec(d), 7)
total_objective = sum(d_vec[i]*d_vec[i]+d_vec[i] for i in chosen)

println("Number of locations selected: ", num_locs)
println("Chosen location indices: ", chosen)
println("Total distance to City Hall: ", total_dist, " miles")
println("Total objective value: ", total_objective)

Number of locations selected: 5
Chosen location indices: [5, 6, 9, 10, 14]
Total distance to City Hall: 5.663774772660714 miles
Total objective value: 14.853060217024849


In [ ]:
num_locs, chosen, total_dist, model = optimize_installations_dist(D, W2, vec(d), 4)
total_objective = sum(d_vec[i]*d_vec[i]+d_vec[i] for i in chosen)

println("Number of locations selected: ", num_locs)
println("Chosen location indices: ", chosen)
println("Total distance to City Hall: ", total_dist, " miles")
println("Total objective value: ", total_objective)

Number of locations selected: 4
Chosen location indices: [6, 9, 10, 34]
Total distance to City Hall: 5.368448838796317 miles
Total objective value: 14.91512325402067


In [31]:
num_locs, chosen, total_dist, model = optimize_installations_dist(D, W2, d_vec, 3)
total_objective = sum(d_vec[i]*d_vec[i]+d_vec[i] for i in chosen)

println("Number of locations selected: ", num_locs)
println("Chosen location indices: ", chosen)
println("Total distance to City Hall: ", total_dist, " miles")
println("Total objective value: ", total_objective)

LoadError: ArgumentError: reducing over an empty collection is not allowed; consider supplying `init` to the reducer

In [9]:
# Calculate energy produced vs needed for each day
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
energy_produced = zeros(7)
energy_needed = percent_demand .* D

println(chosen)

for t in 1:7
    for i in chosen
        # energy_produced[t] += W[i, t] * 0.1
        energy_produced[t] += W2[i, t] * 0.1
    end
end

# Create comparison DataFrame
energy_comparison = DataFrame(
    Day = days,
    Energy_Produced = energy_produced,
    Energy_Needed = energy_needed,
    Surplus = energy_produced .- energy_needed,
    # Coverage_Percent = (energy_produced ./ energy_needed) .* 100
)

println("=" ^ 70)
println("Energy Production vs Demand Analysis")
println("=" ^ 70)
println(energy_comparison)
println()


[5, 6, 9, 10, 14]
Energy Production vs Demand Analysis
7×4 DataFrame
 Row │ Day        Energy_Produced  Energy_Needed  Surplus  
     │ String     Float64          Float64        Float64  
─────┼─────────────────────────────────────────────────────
   1 │ Monday             1215.58        1144.2    71.3789
   2 │ Tuesday            1214.98        1181.74   33.2301
   3 │ Wednesday          1199.82        1191.14    8.6735
   4 │ Thursday           1233.8         1190.01   43.7859
   5 │ Friday             1215.58        1161.43   54.1493
   6 │ Saturday           1215.58        1054.46  161.124
   7 │ Sunday             1215.58        1061.85  153.735



In [10]:
# Calculate total foot traffic per location (sum across all days)
total_foot_traffic = [sum(W2[i, :]) for i in 1:size(W2, 1)]

println(chosen)
# Create DataFrame with indices and total foot traffic
location_traffic = DataFrame(
    Index = 1:size(W2, 1),
    Total_Foot_Traffic = total_foot_traffic
)

# Sort by total foot traffic (descending)
sort!(location_traffic, :Total_Foot_Traffic, rev=true)

println("=" ^ 70)
println("Top 10 Locations by Total Foot Traffic")
println("=" ^ 70)
println(first(location_traffic, 10))

[5, 6, 9, 10, 14]
Top 10 Locations by Total Foot Traffic
10×2 DataFrame
 Row │ Index  Total_Foot_Traffic 
     │ Int64  Float64            
─────┼───────────────────────────
   1 │     6            34608.0
   2 │    10            24220.0
   3 │     9            20345.5
   4 │    24            14568.8
   5 │     8            14199.5
   6 │    20            11077.5
   7 │    33             9224.25
   8 │     7             5005.0
   9 │    36             4739.0
  10 │    34             4641.0


In [11]:
# Create DataFrame with location IDs and distances for selected locations, sorted by distance
selected_locations = DataFrame(
    Index = 1:size(W2, 1),
    Distance_miles = d_vec
)

println(chosen)
# Sort by distance
sort!(selected_locations, :Distance_miles)

println("=" ^ 70)
println("Selected Locations Sorted by Distance to City Hall")
println("=" ^ 70)
println(first(selected_locations, 10))


[5, 6, 9, 10, 14]
Selected Locations Sorted by Distance to City Hall
10×2 DataFrame
 Row │ Index  Distance_miles 
     │ Int64  Float64        
─────┼───────────────────────
   1 │    10        0.357348
   2 │     2        0.380489
   3 │    15        0.662542
   4 │    16        0.662542
   5 │    14        0.693014
   6 │     1        0.854314
   7 │     5        0.854314
   8 │    34        1.252
   9 │     6        1.25263
  10 │    39        1.42072


# Robust Opt

In [34]:
using JuMP
using HiGHS
using LinearAlgebra
using Statistics

percent_demand = 0.00001
watts_converted = 0.1

function optimize_installations_dist_robust(
    D_mean::Vector{Float64},      # Mean demand vector
    D_std::Vector{Float64},        # Standard deviation of demand
    W_mean::Matrix{Float64},       # Mean traffic matrix
    W_std::Matrix{Float64},        # Standard deviation of traffic matrix
    d::Vector{Float64},            # Distance vector
    k = 2.0,              # Uncertainty parameter (k standard deviations)
    max_loc = 1000               # Maximum number of locations
)

    n, T = size(W_mean)  # n locations, T = 7 days
    @assert length(D_mean) == T "Demand vector must have length 7"
    @assert length(D_std) == T "Demand std vector must have length 7"
    @assert size(W_std) == size(W_mean) "W_std must have same size as W_mean"
    @assert length(d) == n "Distance vector d must have length n"

    model = Model(HiGHS.Optimizer)
    set_silent(model)

    @variable(model, x[1:n], Bin) # Binary decision variables

    # Objective: minimize e^distance - 1 for selected locations
    @objective(model, Min, sum((d[i]*d[i] + d[i]) * x[i] for i in 1:n))

    # Constraint: maximum locations can be selected
    @constraint(model, sum(x[i] for i in 1:n) ≤ max_loc)

    # Robust constraint: energy production must meet demand each day
    # For worst-case scenario: minimum energy production ≥ maximum demand
    # Worst-case energy production: W_mean - k*W_std (conservative)
    # Worst-case demand: D_mean + k*D_std (conservative)
    for t in 1:T
        # Robust constraint: must satisfy for worst-case (minimum traffic, maximum demand)
        @constraint(model, 
            sum((W_mean[i, t] - k * W_std[i, t]) * x[i] * watts_converted for i in 1:n) 
            ≥ percent_demand * (D_mean[t] + k * D_std[t])
        )
    end

    optimize!(model) # solve model

    status = termination_status(model)
    if status != MOI.OPTIMAL
        println("Warning: Solver returned status $status")
    end

    chosen_locations = findall(i -> value(x[i]) > 0.5, 1:n)
    total_distance = sum(d[i] for i in chosen_locations)
    num_locations = length(chosen_locations)

    return num_locations, chosen_locations, total_distance, model
end


optimize_installations_dist_robust (generic function with 6 methods)

In [35]:
# Example usage of robust optimization
# Note: In practice, you would compute D_mean, D_std, W_mean, W_std from historical data
# For demonstration, we'll use the current values as means and assume some standard deviations

# Use current D as mean demand
D_mean_robust = D

# Assume 10% coefficient of variation for demand (std = 0.1 * mean)
D_std_robust = 0.1 .* D_mean_robust

# Use current W2 as mean traffic
W_mean_robust = W2

# Assume 15% coefficient of variation for traffic (std = 0.15 * mean)
# Ensure non-negative standard deviations
W_std_robust = max.(0.15 .* W_mean_robust, 0.01)

# Convert distance vector
d_vec_robust = vec(d)

44-element Vector{Float64}:
   0.8543141643882903
   0.3804885933953209
   1.752712422621023
   1.752712422621023
   0.8543141643882903
   1.252628767954841
   2.752846876470099
   3.5368162772536382
   2.50646904367768
   0.3573483661354215
   3.3908562886125555
   4.477706881477856
   1.7849487705970304
   ⋮
   2.5058913746766427
   1.2520026610283743
 571.7516825814232
   8.117528541620022
   1.7959735436002795
   4.553182432440626
   1.420716564322145
   1.420716564322145
   1.9425183775603287
   1.9425183775603287
   3.602769308142643
   3.602769308142643

In [36]:
# Call robust optimization
num_locs_robust, chosen_robust, total_dist_robust, model_robust = 
    optimize_installations_dist_robust(
        D_mean_robust, D_std_robust, 
        W_mean_robust, W_std_robust, 
        d_vec_robust, 
        1 # Set uncertainty parameter k (e.g., k=2.0 means protect against 2 standard deviations)
    )

# Calculate total objective value
total_objective_robust = sum(d_vec_robust[i]*d_vec_robust[i] + d_vec_robust[i] for i in chosen_robust)

println("Robust Optimization Results:")
println("Number of locations selected: ", num_locs_robust)
println("Chosen location indices: ", chosen_robust)
println("Total distance to City Hall: ", total_dist_robust, " miles")
println("Total objective value: ", total_objective_robust)
println()

Robust Optimization Results:
Number of locations selected: 8
Chosen location indices: [2, 5, 6, 9, 10, 20, 33, 34]
Total distance to City Hall: 11.930138114407214 miles
Total objective value: 36.58894197016831



In [ ]:
# Call robust optimization
num_locs_robust, chosen_robust, total_dist_robust, model_robust = 
    optimize_installations_dist_robust(
        D_mean_robust, D_std_robust, 
        W_mean_robust, W_std_robust, 
        d_vec_robust, 
        2 # Set uncertainty parameter k (e.g., k=2.0 means protect against 2 standard deviations)
    )

# Calculate total objective value
total_objective_robust = sum(d_vec_robust[i]*d_vec_robust[i] + d_vec_robust[i] for i in chosen_robust)

println("Robust Optimization Results:")
println("Number of locations selected: ", num_locs_robust)
println("Chosen location indices: ", chosen_robust)
println("Total distance to City Hall: ", total_dist_robust, " miles")
println("Total objective value: ", total_objective_robust)
println()

In [37]:
# Call robust optimization
num_locs_robust, chosen_robust, total_dist_robust, model_robust = 
    optimize_installations_dist_robust(
        D_mean_robust, D_std_robust, 
        W_mean_robust, W_std_robust, 
        d_vec_robust, 
        1, # Set uncertainty parameter k (e.g., k=2.0 means protect against 2 standard deviations)
        7 # set max_loc param
    )

# Calculate total objective value
total_objective_robust = sum(d_vec_robust[i]*d_vec_robust[i] + d_vec_robust[i] for i in chosen_robust)

println("Robust Optimization Results:")
println("Number of locations selected: ", num_locs_robust)
println("Chosen location indices: ", chosen_robust)
println("Total distance to City Hall: ", total_dist_robust, " miles")
println("Total objective value: ", total_objective_robust)
println()

Robust Optimization Results:
Number of locations selected: 7
Chosen location indices: [4, 6, 9, 10, 20, 33, 34]
Total distance to City Hall: 12.448047779244629 miles
Total objective value: 39.304228210237355



In [38]:
# Call robust optimization
num_locs_robust, chosen_robust, total_dist_robust, model_robust = 
    optimize_installations_dist_robust(
        D_mean_robust, D_std_robust, 
        W_mean_robust, W_std_robust, 
        d_vec_robust, 
        1, # Set uncertainty parameter k (e.g., k=2.0 means protect against 2 standard deviations)
        6 # set max_loc param
    )

# Calculate total objective value
total_objective_robust = sum(d_vec_robust[i]*d_vec_robust[i] + d_vec_robust[i] for i in chosen_robust)

println("Robust Optimization Results:")
println("Number of locations selected: ", num_locs_robust)
println("Chosen location indices: ", chosen_robust)
println("Total distance to City Hall: ", total_dist_robust, " miles")
println("Total objective value: ", total_objective_robust)
println()

Robust Optimization Results:
Number of locations selected: 6
Chosen location indices: [6, 8, 9, 10, 20, 34]
Total distance to City Hall: 11.7262602592006 miles
Total objective value: 41.740017651150765

